In [5]:
# auto-load when code changes outside
%load_ext autoreload
%autoreload 2
%load_ext pyinstrument

import sys
sys.path.insert(0,'..')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The pyinstrument extension is already loaded. To reload it, use:
  %reload_ext pyinstrument


In [6]:
# imports 
import torch
import pandas as pd

from src.baseline.vocabulary import Vocabulary
from src.baseline.data_loader import get_loader, get_mean, get_std
from src.baseline.coa_model import get_new_model, init_testing_model,test_model,test_rand_image

In [7]:
# 30 minutes to create those, as it's baseline, i ran it several times and it's the same
vocab = Vocabulary(freq_threshold=1)
vocab.stoi = {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3, 'lion': 4, 'rampant': 5, 'passt': 6, 'guard': 7, 'head': 8, 'lions': 9, 'cross': 10, 'moline': 11, 'patonce': 12, 'eagle': 13, 'doubleheaded': 14, 'eagles': 15, 'a': 16, 'b': 17, 'o': 18, 's': 19, 'g': 20, 'e': 21, 'v': 22, '1': 23, '2': 24, '3': 25, '4': 26, '5': 27, '6': 28, '7': 29, '8': 30, '9': 31, '10': 32, '11': 33, 'border': 34, '&': 35}
vocab.itos = {0: '<PAD>', 1: '<SOS>', 2: '<EOS>', 3: '<UNK>', 4: 'lion', 5: 'rampant', 6: 'passt', 7: 'guard', 8: 'head', 9: 'lions', 10: 'cross', 11: 'moline', 12: 'patonce', 13: 'eagle', 14: 'doubleheaded', 15: 'eagles', 16: 'a', 17: 'b', 18: 'o', 19: 's', 20: 'g', 21: 'e', 22: 'v', 23: '1', 24: '2', 25: '3', 26: '4', 27: '5', 28: '6', 29: '7', 30: '8', 31: '9', 32: '10', 33: '11', 34: 'border', 35: '&'}


## Evaluate a vanilla model on the real data

In [8]:
# REAL_DATASET='/home/salnabulsi/coat-of-arms/data/cropped_coas/out'
# data_location = REAL_DATASET + '/resized'
REAL_DATASET='/Users/salnabulsi/tub/coat-of-arms/data/cropped_coas/out'
data_location = REAL_DATASET + '/images/resized'

test_annotation_psumsq_file  = REAL_DATASET + '/test_real_captions_psumsq.txt'


In [10]:
# Hyperparams
embed_size=300
vocab_size = 36
attention_dim=256
encoder_dim=2048
decoder_dim=512
learning_rate = 3e-4
drop_prob=0.3
ignored_idx = 0
batch_size = 125
NUM_WORKER = 2
freq_threshold = 5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

hyper_params = {'embed_size': embed_size,
                'attention_dim': attention_dim,
                'encoder_dim': encoder_dim,
                'decoder_dim': decoder_dim,
                'vocab_size': vocab_size
              }

pretrained = False
model, optimizer, criterion = get_new_model(hyper_params, learning_rate, ignored_idx, drop_prob, device, pretrained)


## Get size of the model

In [15]:
# from torchinfo import summary
# summary(model)
# Rest net 50

Layer (type:depth-idx)                             Param #
EncoderDecoder                                     --
├─EncoderCNN: 1-1                                  --
│    └─Sequential: 2-1                             --
│    │    └─Conv2d: 3-1                            (9,408)
│    │    └─BatchNorm2d: 3-2                       (128)
│    │    └─ReLU: 3-3                              --
│    │    └─MaxPool2d: 3-4                         --
│    │    └─Sequential: 3-5                        (215,808)
│    │    └─Sequential: 3-6                        (1,219,584)
│    │    └─Sequential: 3-7                        (7,098,368)
│    │    └─Sequential: 3-8                        (14,964,736)
├─DecoderRNN: 1-2                                  --
│    └─Embedding: 2-2                              10,800
│    └─Attention: 2-3                              --
│    │    └─Linear: 3-9                            131,328
│    │    └─Linear: 3-10                           524,544
│    │    └─Linear: 

In [11]:
from torchinfo import summary
summary(model)
# Rest net 18


Layer (type:depth-idx)                             Param #
EncoderDecoder                                     --
├─EncoderCNN: 1-1                                  --
│    └─Sequential: 2-1                             --
│    │    └─Conv2d: 3-1                            (9,408)
│    │    └─BatchNorm2d: 3-2                       (128)
│    │    └─ReLU: 3-3                              --
│    │    └─MaxPool2d: 3-4                         --
│    │    └─Sequential: 3-5                        (147,968)
│    │    └─Sequential: 3-6                        (525,568)
│    │    └─Sequential: 3-7                        (2,099,712)
│    │    └─Sequential: 3-8                        (8,393,728)
├─DecoderRNN: 1-2                                  --
│    └─Embedding: 2-2                              10,800
│    └─Attention: 2-3                              --
│    │    └─Linear: 3-9                            131,328
│    │    └─Linear: 3-10                           524,544
│    │    └─Linear: 3-1

## Init model and run the tests

In [12]:
mean=0.3272075951099396
std=0.3805903494358063
#09-baseline-model-test

In [13]:

test_loader, test_dataset = init_testing_model(test_annotation_psumsq_file, data_location, 
                                               mean, std,
                                               NUM_WORKER,vocab, batch_size, 
                                               device, pin_memory=False,
                                               img_h=620, img_w=600)


In [14]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [15]:
device

device(type='cpu')

In [16]:

test_model(model, criterion,test_loader, test_dataset, vocab_size, device, 'vanilla-logs', True) 


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x512 and 2048x512)

In [19]:
summary(model)

Layer (type:depth-idx)                             Param #
EncoderDecoder                                     --
├─EncoderCNN: 1-1                                  --
│    └─Sequential: 2-1                             --
│    │    └─Conv2d: 3-1                            (9,408)
│    │    └─BatchNorm2d: 3-2                       (128)
│    │    └─ReLU: 3-3                              --
│    │    └─MaxPool2d: 3-4                         --
│    │    └─Sequential: 3-5                        (215,808)
│    │    └─Sequential: 3-6                        (1,219,584)
│    │    └─Sequential: 3-7                        (7,098,368)
│    │    └─Sequential: 3-8                        (14,964,736)
├─DecoderRNN: 1-2                                  --
│    └─Embedding: 2-2                              10,800
│    └─Attention: 2-3                              --
│    │    └─Linear: 3-9                            131,328
│    │    └─Linear: 3-10                           524,544
│    │    └─Linear: 

In [21]:
# torch.cuda.empty_cache()
test_rand_image(model,test_dataset,test_loader,device)

skipping image 249_O G 2 lions passt.jpg, as it does not exist
skipping image 22346_O S cross moline.jpg, as it does not exist
skipping image 5269_G O cross.jpg, as it does not exist


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/salnabulsi/.thesis-py38/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/salnabulsi/.thesis-py38/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/salnabulsi/.thesis-py38/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/salnabulsi/.local/lib/python3.8/site-packages/torchdatasets/_base.py", line 115, in __getitem__
    sample, maps_start = get_sample(self, index, getitem_function)
  File "/home/salnabulsi/.local/lib/python3.8/site-packages/torchdatasets/_base.py", line 95, in get_sample
    sample = original_getitem(self, index)
  File "/home/salna

## Evaluate a vanilla model on the synthetic test data


In [26]:
# data_location =  '../baseline-gen-data/large'
data_location =  '/home/space/datasets/COA/generated-data-api-large'
root_folder_images = data_location + '/images'
test_psumsq_file = data_location + '/test_captions_psumsq.txt'
df = pd.read_csv(test_caption_file)

print("There are {} test images".format(len(df)))

NameError: name 'test_caption_file' is not defined

In [27]:
#Hyperparams
embed_size=300
vocab_size = 36
attention_dim=256
encoder_dim=2048
decoder_dim=512
learning_rate = 3e-4
drop_prob=0.3
ignored_idx = 0
batch_size = 125
NUM_WORKER = 2
freq_threshold = 5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

hyper_params = {'embed_size': embed_size,
                'attention_dim': attention_dim,
                'encoder_dim': encoder_dim,
                'decoder_dim': decoder_dim,
                'vocab_size': vocab_size
              }
pretrained = False
model, optimizer, criterion = get_new_model(hyper_params, learning_rate, ignored_idx, drop_prob, device, pretrained)


In [28]:
test_loader, test_dataset = init_testing_model(test_psumsq_file, root_folder_images, 
                                               mean, std,
                                               NUM_WORKER,vocab,batch_size, 
                                               device, pin_memory=False,
                                               img_h=500, img_w=500)


In [29]:
test_model(model, criterion,test_loader, test_dataset, vocab_size, device, 'vanilla-logs', real_data=False) 

RuntimeError: CUDA error: no kernel image is available for execution on the device

In [30]:
# torch.cuda.empty_cache()
test_rand_image(model,test_dataset,test_loader,device)